<a href="https://colab.research.google.com/github/Sabrina112358/EndPointHealth_Coding_test/blob/main/EndPointHealth_Coding_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coding test
Use the provided admissions, edstays, d_labitems and labevents files from the demo version of MIMIC-IV (documentation available at https://mimic.mit.edu) with a programming language of your choice and answer the following:

1.	What is the percentage of patients admitted to the hospital that have never been to the ED?
2.	For each patient, generate a json with the hospital admission data.
3.	Check the performance of albumin in predicting in-hospital mortality.

  a.	Use only each patient’s first admission to the hospital; 

  b.	Only the Albumin results from the first 24h from hospital admission.


## Loading Libraries

In [1]:
import numpy as np
import pandas as pd
import json
from datetime import datetime
from datetime import timedelta
import plotly.express as px

## Importing data

Subset from the demo version of MIMIC-IV (documentation available at https://mimic.mit.edu)

In [2]:
admission = pd.read_csv("https://raw.githubusercontent.com/Sabrina112358/EndPointHealth_Coding_test/main/data/admissions.csv")
d_labitems = pd.read_csv("https://raw.githubusercontent.com/Sabrina112358/EndPointHealth_Coding_test/main/data/d_labitems.csv")
edstays = pd.read_csv("https://raw.githubusercontent.com/Sabrina112358/EndPointHealth_Coding_test/main/data/edstays.csv")
labevents = pd.read_csv("https://raw.githubusercontent.com/Sabrina112358/EndPointHealth_Coding_test/main/data/labevents.csv")

## Data Exploration

Process for understanding the data types and quantity each table provided.

### Admission

The *admissions* table gives information regarding a patient’s admission to the hospital. Since each unique hospital visit for a patient is assigned a unique `hadm_id`, the *admissions* table can be considered as a definition table for `hadm_id`. Information available includes timing information for admission and discharge, demographic information, the source of the admission, and so on.

---

**Important considerations**

* The data is sourced from the admission, discharge and transfer database from the hospital (often referred to as ‘ADT’ data).

* Organ donor accounts are sometimes created for patients who died in the hospital. These are distinct hospital admissions with very short, sometimes negative lengths of stay. Furthermore, their `deathtime` is frequently the same as the earlier patient admission’s `deathtime`.

---



In [3]:
rows, columns = admission.shape
print("Admission table has", rows, "rows and", columns, "columns.")

Admission table has 275 rows and 15 columns.


In [4]:
# Columns names
print(admission.columns.values)

['subject_id' 'hadm_id' 'admittime' 'dischtime' 'deathtime'
 'admission_type' 'admission_location' 'discharge_location' 'insurance'
 'language' 'marital_status' 'ethnicity' 'edregtime' 'edouttime'
 'hospital_expire_flag']


In [5]:
#display nulls in data within range
admission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   subject_id            275 non-null    int64 
 1   hadm_id               275 non-null    int64 
 2   admittime             275 non-null    object
 3   dischtime             275 non-null    object
 4   deathtime             15 non-null     object
 5   admission_type        275 non-null    object
 6   admission_location    275 non-null    object
 7   discharge_location    233 non-null    object
 8   insurance             275 non-null    object
 9   language              275 non-null    object
 10  marital_status        263 non-null    object
 11  ethnicity             275 non-null    object
 12  edregtime             181 non-null    object
 13  edouttime             181 non-null    object
 14  hospital_expire_flag  275 non-null    int64 
dtypes: int64(3), object(12)
memory usage: 32

In [6]:
# determine the number of null or missing values in each column
admission.isnull().sum()

subject_id                0
hadm_id                   0
admittime                 0
dischtime                 0
deathtime               260
admission_type            0
admission_location        0
discharge_location       42
insurance                 0
language                  0
marital_status           12
ethnicity                 0
edregtime                94
edouttime                94
hospital_expire_flag      0
dtype: int64

In [7]:
admission.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,10004235,24181354,2196-02-24T14:38:00,2196-03-04T14:02:00,NaN,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicaid,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2196-02-24T12:15:00,2196-02-24T17:07:00,0
1,10009628,25926192,2153-09-17T17:08:00,2153-09-25T13:20:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,?,MARRIED,HISPANIC/LATINO,NaN,NaN,0
2,10018081,23983182,2134-08-18T02:02:00,2134-08-23T19:35:00,NaN,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,2134-08-17T16:24:00,2134-08-18T03:15:00,0
3,10006053,22942076,2111-11-13T23:39:00,2111-11-15T17:20:00,2111-11-15T17:20:00,URGENT,TRANSFER FROM HOSPITAL,DIED,Medicaid,ENGLISH,NaN,UNKNOWN,NaN,NaN,1
4,10031404,21606243,2113-08-04T18:46:00,2113-08-06T20:57:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,NaN,NaN,0


### d_labitems
Dimension table for *labevents*; provides a description of all lab items.

d_labitems contains definitions for all `itemid` associated with lab measurements in the MIMIC database. All data in LABEVENTS link to the d_labitems table. Each unique (`fluid`, `category`, `label`) tuple in the hospital database was assigned an `itemid` in this table, and the use of this `itemid` facilitates efficient storage and querying of the data.

Laboratory data contains information collected and recorded in the hospital laboratory database. This includes measurements made in wards within the hospital and clinics outside the hospital. Most concepts in this table have been mapped to LOINC codes, an openly available ontology which facilitates interoperability.

---

**Important considerations**

* Many of the LOINC codes were assigned during a project to standardize the ontology of lab measurements in the MIMIC database. Consequently, the codes were assigned post-hoc, may not be perfect, and may not be present for every lab measurement. We welcome improvements to the present codes or assignment of LOINC codes to unmapped data elements from the community.

---

In [8]:
rows, columns = d_labitems.shape
print("d_labitems table has", rows, "rows and", columns, "columns.")

d_labitems table has 1625 rows and 5 columns.


In [9]:
# Columns names
print(d_labitems.columns.values)

['itemid' 'label' 'fluid' 'category' 'loinc_code']


In [10]:
#display nulls in data within range
d_labitems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1625 entries, 0 to 1624
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   itemid      1625 non-null   int64 
 1   label       1622 non-null   object
 2   fluid       1625 non-null   object
 3   category    1625 non-null   object
 4   loinc_code  267 non-null    object
dtypes: int64(1), object(4)
memory usage: 63.6+ KB


In [11]:
# determine the number of null or missing values in each column
d_labitems.isnull().sum()

itemid           0
label            3
fluid            0
category         0
loinc_code    1358
dtype: int64

In [12]:
d_labitems.head()

,itemid,label,fluid,category,loinc_code
0,52016,Abz,Blood,Blood Gas,NaN
1,52017,"Albumin, Blood",Blood,Blood Gas,NaN
2,50801,Alveolar-arterial Gradient,Blood,Blood Gas,NaN
3,52018,Assist/Control,Blood,Blood Gas,NaN
4,50802,Base Excess,Blood,Blood Gas,NaN


### edstays

The *edstays* table is the primary tracking table for emergency department visits. It provides the time the patient entered the emergency department and the time they left the emergency department.

**Table source:** Emergency department information system.

**Table purpose:** Track patient admissions to the emergency department.

In [13]:
rows, columns = edstays.shape
print("edstays table has", rows, "rows and", columns, "columns.")

edstays table has 210 rows and 5 columns.


In [14]:
# Columns names
print(edstays.columns.values)

['subject_id' 'hadm_id' 'stay_id' 'intime' 'outtime']


In [15]:
#display nulls in data within range
edstays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   subject_id  210 non-null    int64  
 1   hadm_id     162 non-null    float64
 2   stay_id     210 non-null    int64  
 3   intime      210 non-null    object 
 4   outtime     210 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 8.3+ KB


In [16]:
# determine the number of null or missing values in each column
edstays.isnull().sum()

subject_id     0
hadm_id       48
stay_id        0
intime         0
outtime        0
dtype: int64

In [17]:
edstays.head()

,subject_id,hadm_id,stay_id,intime,outtime
0,10014729,23300884.0,32435236,2125-03-19T12:36:00,2125-03-19T16:59:47
1,10004235,24181354.0,36583389,2196-02-24T12:15:00,2196-02-24T17:07:00
2,10026255,22059910.0,30926886,2201-07-07T12:31:00,2201-07-07T19:40:00
3,10026255,20437651.0,36948716,2200-09-17T18:38:00,2200-09-18T00:57:00
4,10007058,22954658.0,35770266,2167-11-07T17:57:00,2167-11-07T20:22:00


### labevents

Laboratory measurements sourced from patient derived specimens.

The *labevents* table stores the results of all laboratory measurements made for a single patient. These include hematology measurements, blood gases, chemistry panels, and less common tests such as genetic assays.

In [18]:
rows, columns = labevents.shape
print("labevents table has", rows, "rows and", columns, "columns.")

labevents table has 105546 rows and 15 columns.


In [19]:
# Columns names
print(labevents.columns.values)

['labevent_id' 'subject_id' 'hadm_id' 'specimen_id' 'itemid' 'charttime'
 'storetime' 'value' 'valuenum' 'valueuom' 'ref_range_lower'
 'ref_range_upper' 'flag' 'priority' 'comments']


In [20]:
#display nulls in data within range
labevents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105546 entries, 0 to 105545
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   labevent_id      105546 non-null  int64  
 1   subject_id       105546 non-null  int64  
 2   hadm_id          79126 non-null   float64
 3   specimen_id      105546 non-null  int64  
 4   itemid           105546 non-null  int64  
 5   charttime        105546 non-null  object 
 6   storetime        104590 non-null  object 
 7   value            95398 non-null   object 
 8   valuenum         93499 non-null   float64
 9   valueuom         89966 non-null   object 
 10  ref_range_lower  87430 non-null   float64
 11  ref_range_upper  87430 non-null   float64
 12  flag             39386 non-null   object 
 13  priority         96255 non-null   object 
 14  comments         17928 non-null   object 
dtypes: float64(4), int64(4), object(7)
memory usage: 12.1+ MB


In [21]:
# determine the number of null or missing values in each column
labevents.isnull().sum()

labevent_id            0
subject_id             0
hadm_id            26420
specimen_id            0
itemid                 0
charttime              0
storetime            956
value              10148
valuenum           12047
valueuom           15580
ref_range_lower    18116
ref_range_upper    18116
flag               66160
priority            9291
comments           87618
dtype: int64

In [22]:
labevents.head()

,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,183881,10014354,29600294.0,1808066,51277,2148-08-16 00:00:00,2148-08-16 01:30:00,15.4,15.4,%,10.5,15.5,NaN,ROUTINE,NaN
1,183883,10014354,29600294.0,1808066,51301,2148-08-16 00:00:00,2148-08-16 01:30:00,20.3,20.3,K/uL,4.0,10.0,abnormal,ROUTINE,NaN
2,183888,10014354,29600294.0,1808066,52167,2148-08-16 00:00:00,2148-08-16 01:30:00,49.7,49.7,fL,35.1,46.3,abnormal,ROUTINE,NaN
3,183870,10014354,29600294.0,1808066,51249,2148-08-16 00:00:00,2148-08-16 01:30:00,31.1,31.1,g/dL,32.0,37.0,abnormal,ROUTINE,NaN
4,183865,10014354,29600294.0,1808066,51222,2148-08-16 00:00:00,2148-08-16 01:30:00,9.2,9.2,g/dL,13.7,17.5,abnormal,ROUTINE,NaN


## Handling data

##### Cleaning Data

In [23]:
# Remove duplicates
admission = admission.drop_duplicates()
d_labitems = d_labitems.drop_duplicates()
edstays = edstays.drop_duplicates()
labevents = labevents.drop_duplicates()

In [24]:
# Function to convert string into a datetime object
def convert_time_t(date):
    return datetime.strptime(date, '%Y-%m-%dT%H:%M:%S')

In [25]:
admission.loc[:, ['admittime']] = admission['admittime'].apply(convert_time_t)
admission.loc[:, ['dischtime']] = admission['dischtime'].apply(convert_time_t)

In [26]:
# Check if there is a organ donor in admission table 
print((admission.admittime == admission.deathtime).value_counts(), '\n')
print(((admission.dischtime - admission.admittime) <= pd.Timedelta('0 days 00:00:00')).value_counts())

False    275
dtype: int64 

False    275
dtype: int64


### 1. What is the percentage of patients admitted to the hospital that have never been to the ED?

In [27]:
# Check which ids are in admission table, but aren't in edstays table
not_in = ~admission.subject_id.isin(edstays.subject_id)
not_in_ed = round(admission[not_in].subject_id.count() * 100 / admission.subject_id.count(),2)

print(f"About {not_in_ed}% of patients admitted to the hospital have never been to the ED.")

About 16.36% of patients admitted to the hospital have never been to the ED.


### 2. For each patient, generate a json with the hospital admission data.

In [28]:
# Function to convert datetime object into a string
def convert_time_string(date):
    return date.strftime('%Y-%m-%d')

Creating a subset of the admission table with the columns to use to generate the *json* and using the function *convert_time_t* to convert the *admittime* into a *datetime* pobject.

In [29]:
sub_df = admission[['subject_id', 'admittime']]
sub_df.loc[:, ['admittime']] = sub_df['admittime'].apply(convert_time_string)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Using pandas function to convert the columns *subject_id* and *admittime* into a json file for each patient.

In [30]:
hospital_admission_data = sub_df.to_json(path_or_buf="/content/hospital_admission_data.json", orient='records')

In [31]:
hospital_admission_data = pd.read_json("/content/hospital_admission_data.json")
print(hospital_admission_data)

     subject_id   admittime
0      10004235  2196-02-24
1      10009628  2153-09-17
2      10018081  2134-08-18
3      10006053  2111-11-13
4      10031404  2113-08-04
..          ...         ...
270    10038992  2187-07-29
271    10008287  2145-09-28
272    10022880  2177-03-12
273    10004457  2141-12-17
274    10004457  2147-12-19

[275 rows x 2 columns]


### Check the performance of albumin in predicting in-hospital mortality.

  a.	Use only each patient’s first admission to the hospital; 

  b.	Only the Albumin results from the first 24h from hospital admission.

Creating a subset from the original *d_lsbitems* to select only Albumin itens.

In [32]:
albumin_items = d_labitems[["itemid", "label"]]

Removing `NaN` values from label in *albumin_items* table.

In [33]:
albumin_items = albumin_items.dropna(subset=['label'])

Selecting rows that contains *albumin* in the label description.

In [34]:
albumin_items = albumin_items[albumin_items['label'].str.contains('albu', case=False)]
print(albumin_items)

     itemid                      label
1     52017             Albumin, Blood
84    51542                  (Albumin)
85    50862                    Albumin
476   51921                  (Albumin)
477   51922             Albumin, Stool
518   51070  Albumin/Creatinine, Urine
519   51069             Albumin, Urine
520   52698             Albumin, Urine
623   51520                  (Albumin)
624   50835           Albumin, Ascites
652   51905                  (Albumin)
653   51046           Albumin, Pleural
683   51805                  (Albumin)
684   51019       Albumin, Joint Fluid
717   51833                    (Albumi
718   51834                  <Albumin>
719   51025        Albumin, Body Fluid
800   51890         Surfactant/Albumin
809   51773                  (Albumin)
810   51774                  <Albumin>
811   51775               Albumin, CSF


Creating a subset from *labevents* table witch contains albumin based on *albumin_items* table created.

In [35]:
is_in = labevents.itemid.isin(albumin_items.itemid)
performance_albumin = labevents[is_in]

Creating a subset from *admission* table.

In [36]:
admission_subset = admission[["subject_id", "hadm_id", "admittime", "dischtime", "deathtime", "edregtime", "edouttime", "hospital_expire_flag"]]

Inner join between *performace_albumin* and *albumin_items* and *admission* tables.

In [37]:
performance_albumin = performance_albumin.merge(albumin_items, on='itemid')
performance_albumin = performance_albumin.merge(admission_subset, on=['subject_id', 'hadm_id'])

Selecting only each patient’s first admission to the hospital.

In [38]:
drop_pacient = admission_subset.drop_duplicates(subset=['subject_id'], keep="first")
is_in = performance_albumin.admittime.isin(drop_pacient.admittime)
performance_albumin =  performance_albumin[is_in]

Selecting  only the Albumin results from the first 24h from hospital admission

In [39]:
def convert_time(date):
    return datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

performance_albumin.loc[:, ['storetime']] = performance_albumin['storetime'].apply(convert_time)
performance_albumin.loc[:, ['charttime']] = performance_albumin['charttime'].apply(convert_time)

In [40]:
performance_albumin['delta_time'] = (performance_albumin.storetime - performance_albumin.admittime)

delta = timedelta(
     days=1,
)

performance_albumin = performance_albumin[performance_albumin.delta_time < delta]

Creating a variable containing the time spend in the hospital.

In [41]:
performance_albumin['hospital_stay'] = (performance_albumin.dischtime - performance_albumin.admittime)

In [42]:
performance_albumin = performance_albumin.dropna(subset=["valuenum"])

In [43]:
performance_albumin.head()

,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,...,comments,label,admittime,dischtime,deathtime,edregtime,edouttime,hospital_expire_flag,delta_time,hospital_stay
4,430293,10035631,20385771.0,67314715,50862,2112-12-04 15:00:00,2112-12-04 16:11:00,4.3,4.3,g/dL,...,NaN,Albumin,2112-12-04 00:00:00,2112-12-27 16:24:00,NaN,NaN,NaN,0,0 days 16:11:00,23 days 16:24:00
50,78810,10005866,27167814.0,53113556,50862,2148-03-11 03:00:00,2148-03-11 04:33:00,2.6,2.6,g/dL,...,NaN,Albumin,2148-03-10 16:16:00,2148-03-21 18:30:00,NaN,2148-03-10T04:46:00,2148-03-10T11:27:00,0,0 days 12:17:00,11 days 02:14:00
63,261058,10020944,29974575.0,77425420,50862,2131-02-27 18:00:00,2131-02-27 19:03:00,3.3,3.3,g/dL,...,NaN,Albumin,2131-02-27 15:34:00,2131-03-13 17:01:00,NaN,2131-02-27T13:16:00,2131-02-27T16:40:00,0,0 days 03:29:00,14 days 01:27:00
101,426954,10035185,22580999.0,40149367,50862,2120-05-12 15:00:00,2120-05-12 16:04:00,4.4,4.4,g/dL,...,NaN,Albumin,2120-05-12 12:53:00,2120-05-17 16:00:00,NaN,NaN,NaN,0,0 days 03:11:00,5 days 03:07:00
106,98055,10007795,25135483.0,84302355,50862,2136-05-05 06:00:00,2136-05-05 07:33:00,3.3,3.3,g/dL,...,NaN,Albumin,2136-05-04 20:20:00,2136-05-12 17:12:00,NaN,NaN,NaN,0,0 days 11:13:00,7 days 20:52:00


#### Determining hospital mortality

In [44]:
total_live, total_death = admission.hospital_expire_flag.value_counts()
hospital_mortality = round(total_death * 100/(total_live + total_death),2)

print(f"The hospital presents a mortality rate of {hospital_mortality}%, considering pacients who were admitted more than once.")

The hospital presents a mortality rate of 5.45%, considering pacients who were admitted more than once.


In [45]:
fig = px.histogram(admission, x="hospital_expire_flag", color="ethnicity")
fig.update_layout(
    title_text="Hospital mortality by ethnicity",
    xaxis_title="Hospital Expire Flag",
    yaxis_title="Count"
)
fig.show()

In [46]:
first_admission = admission.drop_duplicates(subset="subject_id", keep="first")
total_live, total_death = first_admission.hospital_expire_flag.value_counts()
hospital_mortality = round(total_death * 100/(total_live + total_death),2)

print(f"The hospital presents a mortality rate of {hospital_mortality}%, considering only each patient’s first admission to the hospital.")

The hospital presents a mortality rate of 8.0%, considering only each patient’s first admission to the hospital.


In [47]:
fig = px.histogram(first_admission, x="hospital_expire_flag", color="ethnicity")
fig.update_layout(
    title_text="Hospital firt admission mortality by ethnicity",
    xaxis_title="Hospital Expire Flag",
    yaxis_title="Count"
)
fig.show()

#### Determining albumin hospital mortality

In [48]:
first_admission_albumin = performance_albumin.drop_duplicates(subset="subject_id", keep="first")
total_live, total_death = first_admission.hospital_expire_flag.value_counts()
hospital_mortality = round(total_death * 100/(total_live + total_death),2)

print(f"The hospital presents a mortality rate of {hospital_mortality}%, considering only each patient’s first admission \n\
to the hospital and only the Albumin results from the first 24h from hospital admission.")

The hospital presents a mortality rate of 8.0%, considering only each patient’s first admission 
to the hospital and only the Albumin results from the first 24h from hospital admission.


In [49]:
fig = px.bar(first_admission_albumin, x='hospital_expire_flag', y='valuenum', color='label')
fig.update_layout(
    title_text="Hospital albumin mortality by dosage",
    xaxis_title="Hospital Expire Flag",
    yaxis_title="Dosage"
)
fig.show()

In [50]:
fig = px.histogram(first_admission_albumin, x="hospital_expire_flag", color="label")
fig.update_layout(
    title_text="Hospital albumin mortality by label",
    xaxis_title="Hospital Expire Flag",
    yaxis_title="Count"
)
fig.show()